# BSOC as a Convoultional Layer

This work is based on papers by Dr. Gabriel Ferrer from Hendrix College. I will explore the practicality of using Bounded Self-Organizing Clusters (BSOC) as a way to create the first convoultional in a convoultional network. I will be using Tensorflow as a baseline.

## Data:
The data comes from pictures taken on a Galaxy S8 at 1280x720px.
### 7/9/2018
Room: 10 Pictures <br/>
Kitchen: 10 Pictures

## 1: Running a basic test:

In [1]:
## Loading the Data
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))



# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

Files already downloaded and verified
Files already downloaded and verified
  car  ship  ship truck


In [18]:
import torch.nn as nn
import torch.nn.functional as F


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.fc1 = nn.Linear(10 * 5 * 5, 10)
        
    def forward(self, x):
        print(x.size())
        x = F.relu(self.conv1(x))
        print(x.size())
        x = x.view(-1, 10 * 5 * 5)
        x = self.fc1(x)
        return x


# class BSOCNet(nn.Module):
#     def __init__(self):
#         super(BSOCNet, self).__init__()
        
net = ConvNet()

It looks like from [here](https://discuss.pytorch.org/t/defining-a-conv2d-layer-with-fixed-kernel/8684) that we can use F.conv2d to manually define layers [documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d) and [this](https://discuss.pytorch.org/t/pytorch-replace-torch-nn-conv2d-with-torch-nn-functional-conv2d/16596) link might help if we get errors

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([4, 3, 32, 32])
torch.Size([4, 10, 28, 28])


RuntimeError: invalid argument 2: size '[-1 x 250]' is invalid for input with 31360 elements at ..\src\TH\THStorage.c:37

In [ ]:
def view_kernels(tensor):
    width, height, kern_width, kern_height = tensor.shape
    _, axarr = plt.subplots(1 , height)
    print(axarr.shape)
    h = 0
    w = 0
    while ((h * height + w * width) < (height * width)):
        axarr[w].axis('off')
        axarr[w].imshow(tensor[w][h])
        print(tensor[w][h])
        print((h * height + w * width))
        w += 1
        if w >= width:
            h += 1
            w = 0
    
    
count = 0
for m in net.modules():
    if isinstance(m, nn.Conv2d):
        if count == 1:
            print((m.weight.data.numpy().shape))
            view_kernels(m.weight.data.numpy())
        count += 1